In [1]:
import dask.dataframe as dd

# Create a pattern to match all files from 2015 to 2023 ending with 'street.csv'
pattern = '{2015,2016,2017,2018,2019,2020,2021,2022,2023}*street.csv'

# Read the matching CSV files
ddf = dd.read_csv(pattern)


OSError: An error occurred while calling the read_csv method registered to the pandas backend.
Original Message: {2015,2016,2017,2018,2019,2020,2021,2022,2023}*street.csv resolved to no files

In [2]:
import glob

# Generate a list of file paths that match the pattern
file_paths = [f for f in glob.glob('*-street.csv') if any(str(year) in f for year in range(2015, 2024))]

In [8]:
ddf = dd.read_csv(file_paths)

In [10]:
result_pandas_df = ddf.compute()

In [13]:
result_pandas_df.shape

(52909388, 12)

In [14]:
result_pandas_df.columns

Index(['Crime ID', 'Month', 'Reported by', 'Falls within', 'Longitude',
       'Latitude', 'Location', 'LSOA code', 'LSOA name', 'Crime type',
       'Last outcome category', 'Context'],
      dtype='object')

In [15]:
import pandas as pd

lsoa_la_uk = pd.read_csv('lsoa_la_uk.csv') 

In [16]:
lsoa_la_uk.columns

Index(['LSOA code', 'la_code'], dtype='object')

In [17]:
result = pd.merge(result_pandas_df, lsoa_la_uk, on='LSOA code', how='left')


In [20]:
summarised_df = result.groupby(['la_code', 'Month', 'Crime type']).size().reset_index(name='count')

In [21]:
summarised_df

,la_code,Month,Crime type,count
0,E06000001,2015-01,Anti-social behaviour,545
1,E06000001,2015-01,Bicycle theft,9
2,E06000001,2015-01,Burglary,76
3,E06000001,2015-01,Criminal damage and arson,117
4,E06000001,2015-01,Drugs,25
...,...,...,...,...
464593,W06000024,2023-05,Robbery,2
464594,W06000024,2023-05,Shoplifting,70
464595,W06000024,2023-05,Theft from the person,7
464596,W06000024,2023-05,Vehicle crime,27


In [22]:
#summarised_df.to_csv('la_street_data_all.csv', index=False)

In [23]:
summarised_df.to_parquet('la_street_data_all.parquet', index=False)

In [24]:
len(summarised_df.la_code.unique())

339